#In this workbook you will
- Learn about imputation and what it's used for
- Use the KNearestNeighbors algorithm to impute data
- Come up with your own imputation strategy

#Imputation
Imputation is fairly intuitive. For the missing data in our dataset, we're going to replace it with values that come from similar records in our dataset that aren't null.
To do this we're going to use the NearestNeighbors algorithm.

In [1]:
import pandas as pd
import numpy as np
import pylab as pl

In [2]:
df = pd.read_csv("./data/credit-data-post-import.csv")

##Cross Validation
We're going to use the simplest type of cross validation. we'll simply split our data into 2 groups: training and test. we'll use the training set to calibrate our model and then use the test set to  evaluate how effective it is.

In [3]:
is_test = np.random.uniform(0, 1, len(df)) > 0.75
train = df[is_test==False]
test = df[is_test==True]

In [4]:
len(train), len(test)

(112327, 37673)

##Be sure to calibrate the imputation with the training set

In [5]:
from sklearn.neighbors import KNeighborsRegressor

income_imputer = KNeighborsRegressor(n_neighbors=1)

#split our data into 2 groups; data containing nulls and data 
# not containing nulls we'll train on the latter and make
# 'predictions' on the null data to impute monthly_income
train_w_monthly_income = train[train.monthly_income.isnull()==False]
train_w_null_monthly_income = train[train.monthly_income.isnull()==True]

In [6]:
train_w_monthly_income.corr()

,serious_dlqin2yrs,revolving_utilization_of_unsecured_lines,age,number_of_time30-59_days_past_due_not_worse,debt_ratio,monthly_income,number_of_open_credit_lines_and_loans,number_of_times90_days_late,number_real_estate_loans_or_lines,number_of_time60-89_days_past_due_not_worse,number_of_dependents
serious_dlqin2yrs,1.000000,-0.001012,-0.103779,0.126544,-0.002410,-0.023843,-0.028537,0.115168,-0.004382,0.097971,0.045259
revolving_utilization_of_unsecured_lines,-0.001012,1.000000,-0.005104,-0.001659,-0.001518,0.003575,-0.013481,-0.000895,0.003028,-0.001006,0.001274
age,-0.103779,-0.005104,1.000000,-0.046989,-0.000334,0.045096,0.187407,-0.045315,0.064963,-0.041125,-0.205417
number_of_time30-59_days_past_due_not_worse,0.126544,-0.001659,-0.046989,1.000000,-0.001840,-0.011500,-0.044874,0.975814,-0.024293,0.980627,0.001031
debt_ratio,-0.002410,-0.001518,-0.000334,-0.001840,1.000000,-0.034228,0.009362,-0.003003,0.022629,-0.002237,0.011675
monthly_income,-0.023843,0.003575,0.045096,-0.011500,-0.034228,1.000000,0.106582,-0.014628,0.147971,-0.012761,0.070330
number_of_open_credit_lines_and_loans,-0.028537,-0.013481,0.187407,-0.044874,0.009362,0.106582,1.000000,-0.072854,0.426207,-0.062342,0.039774
number_of_times90_days_late,0.115168,-0.000895,-0.045315,0.975814,-0.003003,-0.014628,-0.072854,1.000000,-0.040966,0.989471,-0.006124
number_real_estate_loans_or_lines,-0.004382,0.003028,0.064963,-0.024293,0.022629,0.147971,0.426207,-0.040966,1.000000,-0.034922,0.119733
number_of_time60-89_days_past_due_not_worse,0.097971,-0.001006,-0.041125,0.980627,-0.002237,-0.012761,-0.062342,0.989471,-0.034922,1.000000,-0.007666


In [7]:
train_w_monthly_income.corr().ix[:,5]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


serious_dlqin2yrs                             -0.023843
revolving_utilization_of_unsecured_lines       0.003575
age                                            0.045096
number_of_time30-59_days_past_due_not_worse   -0.011500
debt_ratio                                    -0.034228
monthly_income                                 1.000000
number_of_open_credit_lines_and_loans          0.106582
number_of_times90_days_late                   -0.014628
number_real_estate_loans_or_lines              0.147971
number_of_time60-89_days_past_due_not_worse   -0.012761
number_of_dependents                           0.070330
Name: monthly_income, dtype: float64

In [8]:
cols = ['number_real_estate_loans_or_lines', 'number_of_open_credit_lines_and_loans']
income_imputer.fit(train_w_monthly_income[cols], train_w_monthly_income.monthly_income)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                    weights='uniform')

##Replace the mising values

In [9]:
new_values = income_imputer.predict(train_w_null_monthly_income[cols])


In [10]:
train_w_null_monthly_income['monthly_income'] = new_values
new_values

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


array([5022., 2000., 6143., ..., 5041., 1706., 5041.])

In [11]:
#combine the data back together
train = train_w_monthly_income.append(train_w_null_monthly_income)
len(train)

112327

In [12]:
test['monthly_income_imputed'] = income_imputer.predict(test[cols])
test.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,serious_dlqin2yrs,revolving_utilization_of_unsecured_lines,age,number_of_time30-59_days_past_due_not_worse,debt_ratio,monthly_income,number_of_open_credit_lines_and_loans,number_of_times90_days_late,number_real_estate_loans_or_lines,number_of_time60-89_days_past_due_not_worse,number_of_dependents,monthly_income_imputed
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0,16438.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0,2000.0
11,0,0.018798,51,0,0.531529,6501.0,7,0,2,0,2.0,1000.0
15,0,0.548458,64,0,0.209892,11362.0,7,0,1,0,2.0,7548.0
20,0,0.200923,43,0,0.430046,12300.0,10,0,2,0,0.0,6143.0


In [13]:
test['monthly_income'] = np.where(test.monthly_income.isnull(), test.monthly_income_imputed,
                                  test.monthly_income)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
print(pd.value_counts(train.monthly_income.isnull()))
print(pd.value_counts(test.monthly_income.isnull()))

False    112327
Name: monthly_income, dtype: int64
False    37673
Name: monthly_income, dtype: int64


In [15]:
train.to_csv("./data/credit-data-trainingset.csv", index=False)
test.to_csv("./data/credit-data-testset.csv", index=False)

##Trying your own imputation strategy
Partner with the person next to you and think of other ways you might be able to impute missing valus for the `monthly_income` column.

Things to consider:

- Are `number_of_open_credit_lines_and_loans` and `number_real_estate_loans_or_lines` the best predictors?
- What are some other methods you could use for replacing nulls?
- What about handling outlying values? Should a montly income of $3MM be treated as missing?
- Check out the [scikit-learn docs](http://scikit-learn.org/stable/auto_examples/imputation.html) for more examples
- How might you evaluate the effectiveness of your imputation?

##We just did the following

- Split our data into a training set for building our model and a test set for evaluating its performance
- Used KNearestNeighbors to fill in missing values for `monthly_income`